# This notebook implements the CNN-GloVec Model 

In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt 
import glob
from importlib import reload

import os, sys, re, json, time, datetime, shutil
from common import utils, constants, spell

import tensorflow as tf
import tripadvisor_ds
import visualization

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

/home/emmylau/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1. Load Data

In [2]:
reload(tripadvisor_ds)

input_length = 500
max_bytes = 2**31 - 1

data_file = 'data/tripadvisor_ds.pkl'

if os.path.isfile(data_file):

    bytes_in = bytearray(0)
    input_size = os.path.getsize(data_file)
    with open(data_file, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    ds = pickle.loads(bytes_in)
        
else:
    ds = tripadvisor_ds.TripAdvisor_DS().process(input_length=input_length)
    ds.save(data_file)


In [5]:
labels = pd.DataFrame({'rating': ds.train_labels})
labels.rating.value_counts()

3    413463
2    231827
1    106079
0     90053
Name: rating, dtype: int64

## 2. Set up model Configuations

In [ ]:

model_params = dict(V=ds.vocab.size, 
                    embed_dim=100, 
                    num_classes=len(ds.target_labels),
                    encoder_type='cnn', 
                    filter_sizes = [2,3,4,5], 
                    num_filters=128, 
                    hidden_dims=[1024, 64], 
                    input_length=input_length,
                    dropout_rate=0.5,
                    lr=0.00001, 
                    optimizer='adam', 
                    beta=0.00001)
                    
train_params = dict(batch_size=64, 
                    total_epochs=20, 
                    eval_every=2)



summary_params = dict(chkpt_dir="./tmp/266_cnn_" + datetime.datetime.now().strftime("%Y%m%d-%H%M"))

In [71]:
if os.path.isdir(summary_params['chkpt_dir']):
    shutil.rmtree(summary_params['chkpt_dir'])

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


## 3. Read GloVec Embeddings

In [12]:

embeddings_index = dict()
f = open('vocab/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


#randomly generate unknown and digit embeddings
start_token_embed = np.random.randn(model_params['embed_dim'])
end_token_embed = np.random.randn(model_params['embed_dim'])
unk_token_embed = np.random.randn(model_params['embed_dim'])
digit_embed = np.random.randn(model_params['embed_dim'])

embedding_matrix = np.zeros((ds.vocab.size, model_params['embed_dim']))

count = 0
word=[]
for token, index in ds.vocab.word_to_id.items():
    
    #print(index)
    #print(constants.START_TOKEN)
    
    if token == constants.START_TOKEN:
        embedding_matrix[index] = start_token_embed
    elif token == constants.END_TOKEN:
        embedding_matrix[index] = end_token_embed
    elif token == constants.UNK_TOKEN:
        embedding_matrix[index] = unk_token_embed
    elif token == 'DG':
        embedding_matrix[index] = digit_embed
    else:
        if index > ds.vocab.size - 1:
            print("something is wrong shouldn't be here")
            break
        else:
            embedding_vector = embeddings_index.get(token)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
            else:
                word.append(token)  
                count += 1
                #print(token)
                embedding_matrix[index] = np.random.randn(model_params['embed_dim'])
print(count)


marrtiott
fantastis
zenshin
modeern
rntal
annbiversary
ift18
eugyl/nancy
modern-check
closet-like
subway/l
disapporinting
mommy-daddy
🚿
schi
best..
deliveri
pub/hotel
uncomfor
laser-focused
circ-de-sol
hasile
mouses/rats
33w
options-
location-easy
engageme
DG-DG.
ok.we
story.i
amertania
bellegio
.slighty
atmosphere.we
perfextion
joine
קשוב
imponent
.essence
barmtzvah
responsivenes
seminarfest
maintained.or
timein
'curio
rooms.i
하고싶습니다.항공사의
unglaublich
awesome..definitely
gal-friend
kimpton48
heritag
over-pr
chargesd
go-round
\nsave
c-mas
janowiak
aaaahhhhhhh
stylemore
nickel-an
2brs
visit.i
physion
timesquare
wonderdust
bucchanal
stre
terrace-tastic
distriikt
price/nice
recomment
spaciosus
5-ni
رائع
glitche
place-sorta
sensat
propertygreat
fitout
plus.ac
'funky
bugs-
farewel
re~model
staff.disa
hipsteville
vrooommm
conveinant
hotel.gr
argonaunt
kesly
..big
.fresh
w/teens
non-entertainment
housekeeping/maintenance
.adorable
guarantiees
11th-12
staff/location/creature
confortafle
riving


stay..can
canadianguy
side-
half-t
hgi/midtown
beaonca
to.a
timesqure
soacious
stayuing
5*perfe
waldor
hny
wynn-best
staff.the
-made
pre-remodel
honeymoon..
staff..spac
convenience/large
luxurious-
eally
ordere
immaculate-practical-brand
stay.and
coldes
deco-inspired
friendly*
2015th
all.service
inters
express-wall
wonderfulw
'reas
muchlocation
hotel-6
28-may1
somewha
venetian/las
disappointing-too
experia
unmat
eveywhere
accommadatio
i̇n
average..and
strip.lo
stay.very
sohor
niced
startosphere
heater+hidden
bellme
vication
-n
גדול
loccation
experience..nicest
onewtc
charm..
wynham
vibram\nrelaxing
DGDGDG-DGDGDG
conceirge
s.e.c
chioce
heig
nearstrip
hotelthis
vacations-
masqueradin
💓💓
t-m
retreat-great
tournements
hotel.g
miife
renovated-
hipsterish
4thumbs
rennaissance
overi
up/dropp
wrost
'lack
updated-nee
efficiency/business
expereince
wear-
complety
***just
expected/
reasonabley
'dumped
fitzparick
comendable
comrback
property..huge
harrahs/
in-dining
300/nig
3ni
okay~
florr
nosty
f

comforable
circum
un-beatable
attitude~~~
bathroo
amenitites
platinums
staff—willie
👍🏾👍🏾👍🏾
ny.ideally
reasonble
our2
'cancelon
food..and
15years
yeaa
hotel/condo
💙
bogglin
expirance
wi5
york/midtown-south
..will
old-fas
omni-berkshire
service/hidden
business.fantastic
wowsa
birthday-
defent
moneyno
location.hotel
fran-westin
okay-larg
syaed
fisrt
proclaimer
desaster
city-sized
remodeling-
side-stay
hotel.is
hakone/s
cosino
anyw
dciny
accoun
location/facility
revam
+++++
stayed.u
working/nursing
disregar
cruiseripl
6:30pm
hotsl
soooo-o-o
azzie
buuuuuut
improm
specialy
décent
decór
😍😍
brilliant…apart
beautiful/trendy
tropical-feel
underne
forewa
yeeeessssss
requir
reviews…b
hotel-best
beat-lot
..convenient
trye
misle
fly-drive
envir
hoitel
crocodi
breakfad
crroks
view/average
grandparen
unfrienly
mediocore
attendent
surpising
understaff
✔️DG
nbbi
one-ha
maintainedfro
accrosso
back-packer
beds-great
location+excellent
trip/
klakouski
francisto
ahead**
night/5
entertainmenti
t.i.riffic
pos

eventi
metr
'treat
4-seasons/ritz
knuck
chrismas
diff.
wifiwasher
dispi
delight..great
incaesars
inlove
🏆
unserviced
casino/lame
bi-annua
semi-double
30-may-5
rockn
location/quality
deversity
clean.s
non-smoked
ofdownto
non-tacky
👌
✌
tastef
exceede
mediocracy
billling
uplifitin
elegante/di
room—four
updating/cleaning
suite/sweet
3days
crazi
intercontinenal
tdunbar117
sewards
afgain
hotelall
convienieny
generati
weekendvtrip
/n
accurat
siml
arghhhhhhhh
recoomandable
choise
there-we
twent
famly
..the
beatl
francsc
ameritan
off-s
adequatel
spiri
quieter..
re-anniversary
gorgeou
perfect.you
sejpal
elara-
opulent..massive..fun
doubletre
hotel-adequate
lounge=sham
*everyth
value-fun
side/midtown
espetacul
somegood
disabled/handicap
refrigerat
midtown/hk
staybrid
beresfo
pierr
morni
son-i
floor-vie
hotel-beware
fairmontable
starbu
nocks
extraordina
differnce
unrespectful
auful
heada
snowzilla
surte
aug/sep
inmidst
yrip
colorf
terribe
customer-oriented
chels
10th-
..no
hiddengem
homophobics
ro

organisati
DG/DG-DGDGDG
town..
it-want
holet
hotel.not
manda-everything
perfect.i
square/chinat
titani
unhappy/disappointed
servoce
boutque
visit/vacation
bellperson
친절
ny'ers
staff/stay
*ugh*
staffben
sheratom
roofrop
suks
挑選此酒店是因其房費較大街上的酒店
comfie
in-important
button/lig
dirty..carpet
-wacker
'party
penthoue
receptionist/customer
staff-ever
thursdaycheck
after-visit
experiances
**this
hollywwod
dive..reviews
okay-ish
counld'nt
mostl
employee-
frendliest
1130p
no-chip
housekeeping/
old-scho
experiencd
downtown/strip
newyears
inclu
3*\nhotels
check-ou
immed
swellington
rooms/event
stress-reducer
**appauling
non-smoky
captai
specifi
.right
vegas-go
un-be-li-e-va-ble
aeging
unwilli
absolutey
a-location
-look
pre-9/11.t
rundow
hotel-but
uk.it
edisontimes
non-vegas
-prix
تجربة
nigts
vegaas
signat
cristerna
3-nites
ameaising
DGDGDG-DGDGDG.
elsewhere.if
double-billed
access.the
weekyou
hill..still
awesomenes
average/
navig
hotel/best
mariguana
perfecct
opulen
crashpad
microwavea
staftt
medico

...

visized
nyc.h
room/hotel/casino
freshenin
casino/
sleets
pros-close
trumptastic
vacación
boutiqui
espressomachine
dorm-type
spa/club
nyc-style
stay😎
locaed
ラスベガスの歴史を感じさせる
ah..
decis
suuuper
hotel..given
american*
over-bearing
disapionted
myselfholida
gerva
yprk
cnguyenswallow
raynauds
bed-strip
prime-town
neeede
pool/beach
spanking-clean
silverto
cozy/compact
elar
2yo
it-we
desm
place..been
check-in/check-
fufill
***the
sphynx/sunrise
pizze
luncheoned
그리고
cleanthere
expected.easy
minicay
ew.i
location/rooms
mengeling
great.th
convenitent
issu
aussies..south
26juice
customer-unfriendly
all-a
..walkabl
stay-thought
dimmy
素晴らしいホテル
high-stre
familiness
superbingo
lkna
enterta
toilet/shower
vacation.nightmare
somebo
it'ts
udated
nostolgic
yajaira
comfortabl
resevationist
locationc
tepravel
jacqueia
cofee
jloc
stay/zero
concept-
housekeeping=
3-star.pros
satayed
buliding
rooms-very
location/inconsistent
clim
modene
louge
resort~conveniently
limnice
replaceme
harbor/statue
part3
high-roller
u

worst/best
w/tiny
€DG
sutisfied
room+friendly
hotel/price/location
standardness
casablana
ejbt123
trumpste
coveninet
wounderfull
most-friendly
prison-chic
vnea
fun/parking
bleh
new/renovated
vegaaaas
back.mike
pre-t
thompson/parks
233/ni
dsigwing
resturants
exci
sellina
shopping/theatre
loby
approxi
sidetrip
fransc
closecto
hotel/hostel
amuse/lobby
glitsy
17h30
personnel/
what/how
convention/pleasure
cat-friendly
annhal
wear/a
endin
snowey
casino-basically
opposit
fruit-infused
employees..the
vest-pocket
kids..
vlue
escellent
locstion
expectef
hotel/resort
oct.2nd-oct.
teamlam
w/1
waaaaaayy
billagio
property.pros
'red-eye
wooowwww
ameritania..an
hotel-makes
once-gran
stay.started
services/civility
night-room
adjac
surpises
rush/di
boutiqu
pleaseantly
aweseome
karioki
wanderi
dewhawn
jozh
uswill
most-good
check-in/check
tutts
conditon
start/end
noicse
faboules
flutt
hilto
stayng
hyattthis
housekee
inconsi
asian-insp
-arrival
וא
out-of-the-ordinary
supershuttl
trip2016
movememt
liliputi


conveninet
concert.if
weekend—wish
pstory
soho/tribeca
accomodatons
onvernight
romms
towere
26apr
dingy/dirty
location-staff
efficiency-style
beautful
perne
luggage..sheets
locatrd
here***-
view/great
faciility
starema
rioom
fees..
gross-
4-ni
legen
ipadembassy
soho/c
departi
casab
colaboration
ny-stay
locationshould
desonest
uninspi
casino/hotel
ratzker
bell/lobby
consci
controve
placeand
determi
condencending
well-thought-
experience..
a+++++++++
disab
megabu
🍾
clasd
mysteri
majorit
bay/delano
ehhhhhh
located-
immatu
sister-in-l
inviti
/leisure
aafew
conference/work
comferstbk
t/t
handr
aroundcthe
tripdate
mandaly
lakeshake
singers5252
thereif
£££
ms.mckenzie
miidtown
annivarsery
user-unfriendly
flammingo
haloween
nightbreak
dependab
impolit
newi
rcommend
'stand
accomdati
lesoleil
be-close
ameniti
strengthwhen
bavk
king-si
price-value
ameritina
pool/spa
hotel/location/
downtownhotel
sokay
michelang
management/customer
'winter
..below
cleane
hotel/casi
thisbhotel
buisness/fun
perple
h

designedge
incredib
gpod
vegas-lite
night.it
shoppingbags
pleasently
shoould
hotelro
fleetwoo
place-great
chicagao
love…
harrah´s
pittcon
roll-in
resturaunts
reunio
DGDG/DGDG-DG
rewords
location/new
locationproperty
city-like
rooms.comforta
great.could
smoth
4-seasons
avisor
mid-stay
lovecapetown
'y
4mbs
outstadning
grouptrip
hospitality.we
atrip
destation
once..
providi
traceyh
hyatt-not
.pick
..again
vegas2016
accommations
stron
stay-ca-tion
stay。4
kathalin
burthday
facelift.not
hospedamos
star-
vegas-the
residentialneighborhood
chance-find
4c-
business-stay
madii
kooza
vieww
clean/safe
week.locate
efforted
property-best
phlai
seening
aleays
mindfrea
love.y
ridicolous
sungl
game.the
visit/stay
neighborhood-like
frequ
hotels-
nyease
great~
1150pm
district-amazin
surveyrun
assigne
nghts
34t
DGDGDG.
lakevie
broadwaycon
price/performanc
serivce
fianceee
maximillious
spectucular
johnn
placegreat
traveliers
price=expensive
..construction
sighseeing
location.quee
atmosphere.w
.rude
thoght
f

allaspects
aapex
vintage-style
macgical
viewroom
40/night
hotelwonderful
blue-
pro-guest
dreary..
unknowen
-v
veva
rooms.comfortable
theater-goers
uber/lyfts
croweded
oldish
summer.fel
advace
rooms.we
quarky
manhattan-35th
bathrrom
stingyon
unfortunatly
yet-
non-refund
/things
fast..
winderful
location…
i❤nyny
not-so-little
zoomanity
uninterest
-upper
strande
9yo
hotel.casino
myste
recommandation
wxperience
akcabingirl
madison-
square/theater
stressfree
,this
unrestful
woweeee
cordinators
pickick
hete
camb
crawl.dirty
noicy
bewiched
..fa
fabulousl
jones-crew
ahhh-mazing
dissapponted
..if
registeration
hyb
belleclaire
shmancy
value-oriented
put-ou
park-sleep-fly
firewor
executivge
30/70th
aquariam
stayment
weatgate
centri
nyc/theatre
hotel-even
bellagio-hotel
2nd-6th
service.gambles
-gougers
wedn
hamtoninn
location/old
pleasentely
36526


## 4. Build the Model 

In [72]:
def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    #W_embed_ = tf.get_variable("W_embed", shape=[V, embed_dim], \
    #                           initializer=tf.random_uniform_initializer(-init_scale, init_scale), \
    #                           trainable=True)
    
    W_embed_ = tf.constant(embedding_matrix, dtype=tf.float32, name="W_embed")
    
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_, name="embed_x")
        
    return xs_

def fully_connected_layers(h0_, hidden_dims, activation=tf.nn.relu,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        h_ = tf.layers.dense(h_, hdim, activation=activation, name=("Hidden_%d"%i))
        if dropout_rate > 0:
            h_ = tf.layers.dropout(h_, rate=dropout_rate, training=is_training )
    return h_

def softmax_output_layer(h_, labels_, num_classes):
    
    W_out_ = tf.get_variable("W_out",  shape=[h_.get_shape().as_list()[1], num_classes], \
                               initializer=tf.random_normal_initializer())
    b_out_ = tf.get_variable("b_out", shape=[num_classes])

    logits_ = tf.add(tf.matmul(h_, W_out_), b_out_)
        
    if labels_ is None:
        return None, logits_
    
    with tf.variable_scope("Softmax_Layer"):
        #softmax_ = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels_, logits=logits_, name='softmax')
        softmax_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_, logits=logits_)
        
        loss_ = tf.reduce_mean(softmax_)
    
    return loss_, logits_

def BOW(ids_, V, embed_dim, hidden_dims, dropout_rate=0, is_training=None):
    assert is_training is not None, "is_training must be explicitly set to True or False"

    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim)
     
    sum_xs_ = tf.reduce_sum(xs_, 1)

    h_ = fully_connected_layers(sum_xs_, hidden_dims, \
                           dropout_rate=dropout_rate, is_training=is_training)
    return h_, xs_


def conv_net(ids_, V, embed_dim, filter_sizes, num_filters, hidden_dims, input_length, dropout_rate=0.5, is_training=None):

    assert is_training is not None, "is_training must be explicitly set to True or False"

    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim)

    xs_ = tf.expand_dims(xs_, -1)
        
    pooled_outputs_ = []
    for _, filter_size in enumerate(filter_sizes):
        with tf.name_scope("Conv_MaxPool_%d"%filter_size):
            
            # Convolution Layer
            filter_shape = [filter_size, embed_dim, 1, num_filters]
            W_ = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b_ = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv_ = tf.nn.conv2d(
                xs_,
                W_,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            
            # Activation
            h_ = tf.nn.relu(tf.nn.bias_add(conv_, b_), name="relu")
            
            # Maxpooling 
            pooled_ = tf.nn.max_pool(
                h_,
                ksize=[1, input_length - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],
                padding='VALID',
                name="pool")
            pooled_outputs_.append(pooled_)
            
            variable_summaries(pooled_)

    # Combine all the pooled features and flatten it
    num_filters_total = num_filters * len(filter_sizes)
    h_ = tf.concat(pooled_outputs_, 3)
    h_ = tf.reshape(h_, [-1, num_filters_total])
    
    # fully connected layers
    with tf.variable_scope("FC_Layer"):
        h_ = fully_connected_layers(h_, hidden_dims, is_training = is_training)

    return h_, xs_



In [73]:
tf.reset_default_graph() 

X = tf.placeholder(tf.int32, [None, input_length], name='input_x')
Y = tf.placeholder(tf.int32, [None,], name='input_y')
embedding_placeholder = tf.placeholder(tf.float32, [ds.vocab.size, model_params['embed_dim']])
    

if Y is None:
    one_hot_y_ = None
else:
    one_hot_y_ = tf.one_hot(Y, model_params['num_classes'])


if model_params['encoder_type'] == 'cnn':
    h_, xs_ = conv_net(X, model_params['V'], 
                      model_params['embed_dim'], 
                      model_params['filter_sizes'], 
                      model_params['num_filters'], 
                      model_params['hidden_dims'],
                      model_params['input_length'],
                      is_training=True)
elif model_params['encoder_type'] == 'bow':
    h_, xs_ = BOW(X, model_params['V'], 
                      model_params['embed_dim'],  
                      model_params['hidden_dims'],
                      model_params['dropout_rate'],
                      is_training=True)


with tf.variable_scope("Output_Layer"):
    #ce_loss_, logits_ = softmax_output_layer(h_, one_hot_y_, model_params['num_classes'])
    ce_loss_, logits_ = softmax_output_layer(h_, Y, model_params['num_classes'])
    

with tf.name_scope("Prediction"):
    pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
    pred_max_ = tf.argmax(logits_, 1, name="pred_max")
    predictions_dict = {"proba": pred_proba_, "max": pred_max_}

with tf.variable_scope("Regularization"):
    l2_penalty_ = tf.nn.l2_loss(xs_)  # l2 loss on embeddings
    for var_ in tf.trainable_variables():
        if "Embedding_Layer" in var_.name:
            continue
        l2_penalty_ += tf.nn.l2_loss(var_)
    l2_penalty_ *= model_params['beta']  # scale by regularization strength
    tf.summary.scalar('l2_penalty', l2_penalty_)
    regularized_loss_ = ce_loss_ + l2_penalty_
    tf.summary.scalar('regularized_loss', regularized_loss_)

with tf.variable_scope("Training"):
    if model_params['optimizer'] == 'adagrad':
        optimizer_ = tf.train.AdagradOptimizer(model_params['lr'])
    elif  model_params['optimizer'] == 'adam':
        optimizer_ = tf.train.AdamOptimizer(model_params['lr'])
    else:
        optimizer_ = tf.train.GradientDescentOptimizer(model_params['lr'])
    train_op_ = optimizer_.minimize(regularized_loss_,
                    global_step=tf.train.get_global_step())


with tf.name_scope("Evaluation"):
    cross_entropy_loss_ = tf.metrics.mean(ce_loss_)
    #accuracy_ = tf.metrics.accuracy(Y, pred_max_)
    
    correct_pred_ = tf.equal(tf.argmax(logits_, 1), tf.argmax(one_hot_y_, 1))
    #correct_pred_ = tf.equal(tf.argmax(logits_, 1), Y)
    accuracy_ = tf.reduce_mean(tf.cast(correct_pred_, tf.float32))

    eval_metrics = {"cross_entropy_loss": cross_entropy_loss_, "accuracy": accuracy_}
    
    tf.summary.scalar('cross_entropy', ce_loss_)
    tf.summary.scalar('accuracy', accuracy_)



In [74]:
'''
saver = tf.train.Saver()

with tf.Session() as sess:
  # Restore variables from disk.
    saver.restore(sess, "./tmp/model_bow_test.ckpt")
'''

'\nsaver = tf.train.Saver()\n\nwith tf.Session() as sess:\n  # Restore variables from disk.\n    saver.restore(sess, "./tmp/model_bow_test.ckpt")\n'

## 6. Train the model 

In [75]:


# start session
sess = tf.Session()
graph = tf.get_default_graph()

# Tensorboard - Visualize graph 
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(summary_params['chkpt_dir'] + '/train', sess.graph)
test_writer = tf.summary.FileWriter(summary_params['chkpt_dir'] + '/test')

print("tensorboard --logdir={}/train".format(summary_params['chkpt_dir']))
print("tensorboard --logdir={}/test".format(summary_params['chkpt_dir']))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

# Run the initializer
sess.run(init)
sess.run(init_l)

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)



for i in range(train_params['total_epochs']):
    t0 = time.time()

    train_batches = 1
    train_accuracy = 0.0
    
    for (bx, by) in utils.multi_batch_generator(train_params['batch_size'], \
                                        ds.padded_train_features, ds.train_labels):

        summary, batch_loss, _, batch_accuracy = sess.run(
            [merged, regularized_loss_, train_op_, accuracy_], feed_dict={X: bx, Y: by})
        
        train_batches +=1
        train_accuracy += batch_accuracy
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean

        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 25 == 0):
            print("{:5,} examples, moving-average loss {:.2f}, train accuracy {:.2f}"\
                  .format(total_examples, loss_ema, train_accuracy/train_batches))    
            
        train_writer.add_summary(summary, total_batches)

    print("Completed {} epoch in {:s}".format(i, utils.pretty_timedelta(since=t0)))
    
    train_accuracy = train_accuracy/train_batches
    print("Train accurary:{:.5f}".format(train_accuracy))
    
    validate_batches = 1
    validate_accuracy = 0.0
    for (vx, vy) in utils.multi_batch_generator(train_params['batch_size'], \
                                            ds.padded_validate_features, ds.validate_labels):

        summary, batch_accuracy = sess.run([merged, accuracy_], feed_dict={X: vx, Y: vy})

        validate_batches +=1
        validate_accuracy += batch_accuracy

        test_writer.add_summary(summary, total_batches + validate_batches)

    validate_accuracy = validate_accuracy/validate_batches
    print("Validate accuracy:{:.5f}".format(validate_accuracy))
        

tensorboard --logdir=./tmp/266_cnn_20181207-1306/train
tensorboard --logdir=./tmp/266_cnn_20181207-1306/test
1,600 examples, moving-average loss 12.96, train accuracy 0.32
3,200 examples, moving-average loss 13.04, train accuracy 0.36
4,800 examples, moving-average loss 12.96, train accuracy 0.38
6,400 examples, moving-average loss 12.90, train accuracy 0.39
8,000 examples, moving-average loss 12.95, train accuracy 0.40
9,600 examples, moving-average loss 12.78, train accuracy 0.40
11,200 examples, moving-average loss 12.75, train accuracy 0.41
12,800 examples, moving-average loss 12.82, train accuracy 0.41
14,400 examples, moving-average loss 12.72, train accuracy 0.42
16,000 examples, moving-average loss 12.79, train accuracy 0.42
17,600 examples, moving-average loss 12.74, train accuracy 0.42
19,200 examples, moving-average loss 12.73, train accuracy 0.42
20,800 examples, moving-average loss 12.80, train accuracy 0.42
22,400 examples, moving-average loss 12.64, train accuracy 0.42
2

203,200 examples, moving-average loss 12.37, train accuracy 0.48
204,800 examples, moving-average loss 12.33, train accuracy 0.48
206,400 examples, moving-average loss 12.43, train accuracy 0.48
208,000 examples, moving-average loss 12.37, train accuracy 0.48
209,600 examples, moving-average loss 12.38, train accuracy 0.49
211,200 examples, moving-average loss 12.42, train accuracy 0.49
212,800 examples, moving-average loss 12.45, train accuracy 0.49
214,400 examples, moving-average loss 12.42, train accuracy 0.49
216,000 examples, moving-average loss 12.42, train accuracy 0.49
217,600 examples, moving-average loss 12.40, train accuracy 0.49
219,200 examples, moving-average loss 12.44, train accuracy 0.49
220,800 examples, moving-average loss 12.40, train accuracy 0.49
222,400 examples, moving-average loss 12.33, train accuracy 0.49
224,000 examples, moving-average loss 12.34, train accuracy 0.49
225,600 examples, moving-average loss 12.39, train accuracy 0.49
227,200 examples, moving-

406,400 examples, moving-average loss 12.36, train accuracy 0.51
408,000 examples, moving-average loss 12.38, train accuracy 0.51
409,600 examples, moving-average loss 12.35, train accuracy 0.51
411,200 examples, moving-average loss 12.26, train accuracy 0.51
412,800 examples, moving-average loss 12.32, train accuracy 0.51
414,400 examples, moving-average loss 12.35, train accuracy 0.52
416,000 examples, moving-average loss 12.31, train accuracy 0.52
417,600 examples, moving-average loss 12.31, train accuracy 0.52
419,200 examples, moving-average loss 12.29, train accuracy 0.52
420,800 examples, moving-average loss 12.30, train accuracy 0.52
422,400 examples, moving-average loss 12.26, train accuracy 0.52
424,000 examples, moving-average loss 12.31, train accuracy 0.52
425,600 examples, moving-average loss 12.26, train accuracy 0.52
427,200 examples, moving-average loss 12.30, train accuracy 0.52
428,800 examples, moving-average loss 12.24, train accuracy 0.52
430,400 examples, moving-

609,600 examples, moving-average loss 12.22, train accuracy 0.53
611,200 examples, moving-average loss 12.21, train accuracy 0.53
612,800 examples, moving-average loss 12.27, train accuracy 0.53
614,400 examples, moving-average loss 12.30, train accuracy 0.53
616,000 examples, moving-average loss 12.24, train accuracy 0.53
617,600 examples, moving-average loss 12.18, train accuracy 0.53
619,200 examples, moving-average loss 12.30, train accuracy 0.53
620,800 examples, moving-average loss 12.27, train accuracy 0.53
622,400 examples, moving-average loss 12.28, train accuracy 0.53
624,000 examples, moving-average loss 12.19, train accuracy 0.53
625,600 examples, moving-average loss 12.15, train accuracy 0.53
627,200 examples, moving-average loss 12.25, train accuracy 0.54
628,800 examples, moving-average loss 12.26, train accuracy 0.54
630,400 examples, moving-average loss 12.32, train accuracy 0.54
632,000 examples, moving-average loss 12.33, train accuracy 0.54
633,600 examples, moving-

812,800 examples, moving-average loss 12.18, train accuracy 0.55
814,400 examples, moving-average loss 12.12, train accuracy 0.55
816,000 examples, moving-average loss 12.16, train accuracy 0.55
817,600 examples, moving-average loss 12.22, train accuracy 0.55
819,200 examples, moving-average loss 12.18, train accuracy 0.55
820,800 examples, moving-average loss 12.21, train accuracy 0.55
822,400 examples, moving-average loss 12.19, train accuracy 0.55
824,000 examples, moving-average loss 12.19, train accuracy 0.55
825,600 examples, moving-average loss 12.21, train accuracy 0.55
827,200 examples, moving-average loss 12.26, train accuracy 0.55
828,800 examples, moving-average loss 12.17, train accuracy 0.55
830,400 examples, moving-average loss 12.18, train accuracy 0.55
832,000 examples, moving-average loss 12.17, train accuracy 0.55
833,600 examples, moving-average loss 12.16, train accuracy 0.55
835,200 examples, moving-average loss 12.22, train accuracy 0.55
836,800 examples, moving-

...

16,035,850 examples, moving-average loss 11.99, train accuracy 0.72
16,037,450 examples, moving-average loss 11.96, train accuracy 0.72
16,039,050 examples, moving-average loss 11.90, train accuracy 0.72
16,040,650 examples, moving-average loss 11.89, train accuracy 0.72
16,042,250 examples, moving-average loss 11.98, train accuracy 0.72
16,043,850 examples, moving-average loss 11.86, train accuracy 0.72
16,045,450 examples, moving-average loss 11.94, train accuracy 0.72
16,047,050 examples, moving-average loss 11.91, train accuracy 0.72
16,048,650 examples, moving-average loss 11.97, train accuracy 0.72
16,050,250 examples, moving-average loss 11.93, train accuracy 0.72
16,051,850 examples, moving-average loss 11.96, train accuracy 0.72
16,053,450 examples, moving-average loss 11.95, train accuracy 0.72
16,055,050 examples, moving-average loss 11.87, train accuracy 0.72
16,056,650 examples, moving-average loss 11.91, train accuracy 0.72
16,058,250 examples, moving-average loss 11.92, 

16,229,450 examples, moving-average loss 11.94, train accuracy 0.72
16,231,050 examples, moving-average loss 11.86, train accuracy 0.72
16,232,650 examples, moving-average loss 11.92, train accuracy 0.72
16,234,250 examples, moving-average loss 11.86, train accuracy 0.72
16,235,850 examples, moving-average loss 11.98, train accuracy 0.72
16,237,450 examples, moving-average loss 12.05, train accuracy 0.72
16,239,050 examples, moving-average loss 11.94, train accuracy 0.72
16,240,650 examples, moving-average loss 11.96, train accuracy 0.72
16,242,250 examples, moving-average loss 11.92, train accuracy 0.72
16,243,850 examples, moving-average loss 11.94, train accuracy 0.72
16,245,450 examples, moving-average loss 11.90, train accuracy 0.72
16,247,050 examples, moving-average loss 11.99, train accuracy 0.72
16,248,650 examples, moving-average loss 11.94, train accuracy 0.72
16,250,250 examples, moving-average loss 11.88, train accuracy 0.72
16,251,850 examples, moving-average loss 11.99, 

16,423,050 examples, moving-average loss 11.92, train accuracy 0.72
16,424,650 examples, moving-average loss 11.89, train accuracy 0.72
16,426,250 examples, moving-average loss 11.87, train accuracy 0.72
16,427,850 examples, moving-average loss 11.89, train accuracy 0.72
16,429,450 examples, moving-average loss 12.01, train accuracy 0.72
16,431,050 examples, moving-average loss 11.98, train accuracy 0.72
16,432,650 examples, moving-average loss 11.96, train accuracy 0.72
16,434,250 examples, moving-average loss 11.92, train accuracy 0.72
16,435,850 examples, moving-average loss 11.97, train accuracy 0.72
16,437,450 examples, moving-average loss 11.95, train accuracy 0.72
16,439,050 examples, moving-average loss 11.99, train accuracy 0.72
16,440,650 examples, moving-average loss 11.87, train accuracy 0.72
16,442,250 examples, moving-average loss 12.01, train accuracy 0.72
16,443,850 examples, moving-average loss 11.95, train accuracy 0.72
16,445,450 examples, moving-average loss 11.99, 

16,616,650 examples, moving-average loss 11.97, train accuracy 0.72
16,618,250 examples, moving-average loss 12.00, train accuracy 0.72
16,619,850 examples, moving-average loss 11.99, train accuracy 0.72
16,621,450 examples, moving-average loss 11.83, train accuracy 0.72
16,623,050 examples, moving-average loss 11.86, train accuracy 0.72
16,624,650 examples, moving-average loss 11.90, train accuracy 0.72
16,626,250 examples, moving-average loss 11.89, train accuracy 0.72
16,627,850 examples, moving-average loss 11.96, train accuracy 0.72
16,629,450 examples, moving-average loss 12.00, train accuracy 0.72
16,631,050 examples, moving-average loss 11.93, train accuracy 0.72
16,632,650 examples, moving-average loss 12.00, train accuracy 0.72
16,634,250 examples, moving-average loss 11.93, train accuracy 0.72
16,635,850 examples, moving-average loss 11.95, train accuracy 0.72
16,637,450 examples, moving-average loss 11.96, train accuracy 0.72
16,639,050 examples, moving-average loss 11.94, 

16,810,250 examples, moving-average loss 11.96, train accuracy 0.72
16,811,850 examples, moving-average loss 11.99, train accuracy 0.72
16,813,450 examples, moving-average loss 11.99, train accuracy 0.72
16,815,050 examples, moving-average loss 11.92, train accuracy 0.72
16,816,650 examples, moving-average loss 11.88, train accuracy 0.72
16,818,250 examples, moving-average loss 11.96, train accuracy 0.72
16,819,850 examples, moving-average loss 11.93, train accuracy 0.72
16,821,450 examples, moving-average loss 12.04, train accuracy 0.72
16,823,050 examples, moving-average loss 11.91, train accuracy 0.72
16,824,650 examples, moving-average loss 11.92, train accuracy 0.72
16,826,250 examples, moving-average loss 11.90, train accuracy 0.72
16,827,850 examples, moving-average loss 11.95, train accuracy 0.72
Completed 19 epoch in 0:39:33
Train accurary:0.71986
Validate accuracy:0.70797


## 7. Evaluate the Model 

In [76]:
test_batches = 1
test_accuracy = 0.0
test_pred_y = []

for (tx, ty) in utils.multi_batch_generator(train_params['batch_size'], \
                                        ds.padded_test_features, ds.test_labels):

    batch_accuracy, pred_max = sess.run([accuracy_, pred_max_], feed_dict={X: tx, Y: ty})

    test_batches +=1
    test_accuracy += batch_accuracy
    test_pred_y.append(pred_max.tolist())

test_accuracy = test_accuracy/test_batches
print("Test accuracy:{}".format(test_accuracy))

pred_y = [y for x in test_pred_y for y in x]

Test accuracy:0.7057087912192711


In [172]:
save_path = saver.save(sess, "./tmp/model_bow_test.ckpt")


In [77]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ds.test_labels, pred_y)

array([[ 8909,  1529,   403,   115],
       [ 3007,  5506,  4034,   484],
       [  591,  2195, 16346,  9545],
       [  216,   267,  8140, 42592]])

In [56]:
#op = sess.graph.get_operations()
#[m.values() for m in op]



## 8. Implement Integrated Gradients 

In [78]:
def get_gradients(sess, graph, review):
    
    with graph.as_default():

        # get prediction 
        pred_prob_ = graph.get_tensor_by_name('Prediction/pred_proba:0')
        pred_prob = sess.run([pred_prob_], feed_dict={X:[review], Y:[0]})
        
        # get gradient     
        input_y_ = graph.get_tensor_by_name('input_y:0')[0]
        embed_x_ = graph.get_tensor_by_name('Embedding_Layer/embed_x:0')
        label_prediction_ = graph.get_tensor_by_name('Prediction/pred_proba:0')[:, input_y_]
        grads_ = tf.gradients(label_prediction_, embed_x_)[0]
        
        embed_x, grads = sess.run([embed_x_, grads_], feed_dict={X:[review], Y:[np.argmax(pred_prob)]})
        return grads * embed_x

    

In [79]:
def get_integrated_gradients_new(sess, graph, review, steps = 50):

    empty_review = np.zeros(input_length)
    
    with graph.as_default():

        # get embedding and prediction 
        embed_x_ = graph.get_tensor_by_name('Embedding_Layer/embed_x:0')
        pred_prob_ = graph.get_tensor_by_name('Prediction/pred_proba:0')
        embed_x, pred_y, pred_prob = sess.run([embed_x_, pred_max_, pred_prob_], feed_dict={X:[review], Y:[4]})
          
        # empty embedding 
        empty_embed_x, empty_pred_prob = sess.run([embed_x_, pred_prob_], feed_dict={X:[empty_review], Y:[4]})
            
            
        # get integrated gradient 
        input_y_ = graph.get_tensor_by_name('input_y:0')[0]
        label_prediction_ = graph.get_tensor_by_name('Prediction/pred_proba:0')[:, input_y_]
        grads_ = tf.gradients(label_prediction_, embed_x_)[0]

        
        all_grads = []
        for i in range(0, steps + 1):
            _, grads = sess.run([label_prediction_, grads_], feed_dict={embed_x_:(empty_embed_x + (embed_x - empty_embed_x)*(float(i)/steps)), Y:pred_y})
            all_grads.append(grads)
        
        integrated_grads = np.average(all_grads[:-1], axis=0) * embed_x
        return integrated_grads, pred_y[0], pred_prob, empty_pred_prob

### 8.1 Supporting class to save attributions 

In [80]:
class AttributionResult(object):
    def __init__(self, filename):
        self.filename = filename
        self.html_text = "<html><body>"
    
    def append(self, text):
        self.html_text += text
    
    def write(self, index):
        self.html_text += "</body></html>"
        
        with open("result/{}_{}.html".format(self.filename, index), "w") as the_file:
            the_file.write(self.html_text)
            
        self.html_text = "<html><body>"
        
class AttributionCount(object):
    def __init__(self, filename):
        self.filename = filename
        self.word_count = []
        
    def append(self, token):
        self.word_count.append(token)
    
    def write(self, index):
        df = pd.DataFrame.from_records(self.word_count)
        df.columns = ['pred', 'target', 'vocab', 'attr']
        df.attr.round(5)
        df.to_csv("result/{}_{}.csv".format(self.filename, index))
    
        self.word_count = []
        

### 8.2 Visualize the attributions

In [ ]:
reload(visualization)

file_suffix = "cnn_glovec_d{}_h1024_64_epoch{}".format(model_params['embed_dim'], \
                                   train_params['total_epochs'])

corrHigh = AttributionResult("CorrHigh_{}".format(file_suffix))
corrLow = AttributionResult("CorrLow_{}".format(file_suffix))
incorrHigh = AttributionResult("InCorrHigh_{}".format(file_suffix))
incorrLow = AttributionResult("InCorrLow_{}".format(file_suffix))
word_counts = AttributionCount("wordcounts_{}".format(file_suffix))


for i in range(0, len(ds.test_labels)):
    gradients = get_gradients(sess, sess.graph, ds.padded_test_features[i])
    #integrated_gradients, pred_y, pred_proba = get_integrated_gradients(sess, sess.graph, ds.padded_test_features[i])
    integrated_gradients, pred_y, pred_proba, empty_pred_proba = get_integrated_gradients_new(sess, sess.graph, ds.padded_test_features[i])
   
    
    #print(np.matrix(gradients).sum())
    print(np.matrix(integrated_gradients).sum())
    print(pred_proba[0][pred_y])
    print(empty_pred_proba[0][pred_y])

    pred_text = "predicted = {}, actual = {}".format(pred_y+2, ds.test_labels[i]+2)
    print("{}, {}".format(i, pred_text))

    #visualization.visualize_token_attrs(ds.vocab.ids_to_words(ds.test_features[i])[:input_length], np.matrix(gradients).sum(axis=1))
    html_text = visualization.visualize_token_attrs(
            ds.vocab.ids_to_words(ds.test_features[i])[:input_length], 
            np.matrix(integrated_gradients).sum(axis=1), 
            pred_y+2, ds.test_labels[i]+2, word_counts)

    text = "</br>" + pred_text + "</br>" + html_text
    
    if (pred_y == ds.test_labels[i]):
        if (pred_y >= 2):
            corrHigh.append(text)
        else:
            corrLow.append(text)
    else:
        if (ds.test_labels[i] >= 2):
            incorrHigh.append(text)
        else:
            incorrLow.append(text)
    
    if (i % 1000) == 0:
        corrHigh.write(i)
        corrLow.write(i)
        incorrHigh.write(i)
        incorrLow.write(i)
        
        word_counts.write(i)
